# **Основные этапы работы:**
## Этап 0. Установка и настройка оболочки для работы с языком Python

В лабораторной работе использовалсь среда разработки VS Code с расширением Juputer <u>version: 2024.2.0</u>.

Загрузка необходимых библиотек для выполнения лаборатрных работ:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import  Embedding, GRU, Dense
from keras.models import  Sequential
from keras.preprocessing import sequence
from keras.optimizers import RMSprop
from keras import layers
import os

## Этап 1. Построение рекуррентной нейронной сети для распознавания эмоциональной окраски отзывов из базы данных IMDB.

**Целью данного этапа** лабораторной работы является создание рекуррентного бинарного классификатора эмоциональной окраски отзывов из набора данных IMDB. Пошаговая реализация поставленной цели включает.

Вариант этапа 1
| № | Количество рекуррентных слоев | Размерность векторного представления | Количество нейронов на слое | Рекуррентный слой |
| :-: | :-: | :-: | :-: | :-: |
| 3 | 1 | 32 | 128 | GRU |

 Пошаговая реализация поставленной цели включает:
 
    1. Загрузка набора данных IMDB:
База данных состоит из 50000 отзывов к кинолентам в интернет-базе (Internet Movie Database). Набор разбит на 25000 обучающих и 25 000 контрольных отзывов, каждый набор на 50 % состоит из отрицательных и на 50 % из положительных отзывов. Набор данных IMDB поставляется в составе Keras. Набор готов к использованию: отзывы (последовательности слов) преобразованы в последовательности целых чисел, каждое из которых определяет позицию слова в словаре.

In [2]:
from tensorflow.keras.datasets import imdb

Загрузка данных осуществляется из библиотеки Keras. Аргумент при загрузке указывает, что в обучающих данных будет сохранено только 10000 слов, наиболее часто встречающихся в обучающем наборе. Данные изначально разделены на тренировочные и тестовые в соотношении 1:1. Размер обучающего набора составляет 25000 экземпляров.

    2. Создание векторного представления слов из набора данных;
    3. Разделение данных на обучающий и тестовый наборы:
Аргумент _num_words=max_features_ означает, что в обучающих данных будет сохранено только 10000 слов, наиболее часто встречающихся в обучающем наборе отзывов, остальные слова будут отброшены.

In [3]:
max_features = 10000
maxlen = 500
(input_train, y_train), (input_test, y_test) = imdb.load_data(num_words=max_features)

Посмотрим, что из себя представляют данные на примере первого отзыва:

In [5]:
input_train[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 2,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 2,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 2,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,
 5535,
 18,

Переменные _input_train_ и _input_test_ — это списки отзывов; каждый отзыв — это список индексов слов (кодированное представление последовательности слов). Для декодирования данных и получения текста воспользуемся вспомогательной функцией:

In [8]:
index = imdb.get_word_index()

reverse_index = dict([(value,key) for (key, value) in index.items()])
decoder = ' '.join([reverse_index.get(i-3,'.') for i in input_train[0]])
print(decoder)

. this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert . is an amazing actor and now the same being director . father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for . and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also . to the two little boy's that played the . of norman and paul they were just brilliant children are often left out of the . list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you thi

In [9]:
y_train

array([1, 0, 0, ..., 0, 1, 0], dtype=int64)

Переменные _y_train_ и _y_test_ — это списки нулей и единиц, где нули соответствуют отрицательным отзывам, а единицы — положительным.

    4. Подготовка данных для передачи в нейронную сеть
Преобразуем списки целых чисел в двумерный тензор с целыми числами и с формой (образцы, максимальная_длина) с помощью функции _pad_sequences_:

In [6]:
input_train = sequence.pad_sequences(input_train, maxlen=maxlen)
input_test = sequence.pad_sequences(input_test, maxlen=maxlen)
input_train

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]])

    5. Конструирование сети: создание сети из слоя Embedding и рекуррентных слоев заданного типа в соответствии с вариантом;
    6. Настройка оптимизатора с выбором функции потерь и метрики качества. Число эпох принять от 30 до 60:

С помощью функции _Sequential()_ создаём новую модель. Создание модели реализовано при помощи функции _create_model_, она включает в себя конструирование модели, настройку оптимизатора и выбор функций потерь и качества. Первый слой, _Embedding(num_words, 32, input_length=maxlen)_, слой встраивания, который преобразует индексы слов в dense-векторы размерности 32. _num_words_ - это количество уникальных слов в словаре, а _maxlen_ - максимальная длинна входных последовательностей. Второй слой _GRU_, рекуррентный слой c 128 нейронами. Последний слой состоит из одного нейрона с функцией активации _sigmoid_, на выходе получаем скалярное значение в диапозоне между 0 и 1, представляющее собой вероятность.

Слой Embedding это как словарь, отображающий целочисленные индексы (обозначающие конкретные слова) в плотные векторы. Он принимает целые числа на входе, отыскивает их во внутреннем словаре и возвращает соответствующие векторы. Это эффективная операция поиска в словаре (рис. 1). Слой Embedding получает на входе двумерный тензор с целыми числами, каждый элемент которого является последовательностью целых чисел. Он может работать с последовательностями разной длины. Этот слой возвращает трехмерный тензор с вещественными числами и с формой (образцы, длина_последовательности, размерность_векторного_представления). Такой трехмерный тензор можно затем обработать слоем RNN или одномерным сверточным слоем. При создании слоя Embedding, его веса (внутренний словарь векторов токенов) инициализируются случайными значениями. В процессе обучения векторы слов постепенно корректируются посредством обратного распространения ошибки, и пространство превращается в структурированную модель, пригодную к использованию. После полного обучения пространство векторов приобретет законченную структуру, специализированную под решение конкретной задачи.

<center><img src="3.1.6.png"></center>
<center>Рис. 1. Слой Embedding</center>

Рекуррентная нейронная сеть (Recurrent Neural Network, RNN) — это разновидность нейронной сети, имеющей внутренний цикл (рис. 2). Она обрабатывает последовательность, перебирая ее элементы и сохраняя состояние, полученное при обработке предыдущих элементов. Сеть RNN сбрасывает состояние между обработкой двух разных, независимых последовательностей (таких, как два разных отзыва из IMDB), поэтому одна последовательность все еще интерпретируется как единый блок данных: единственный входной пакет. Однако блок данных обрабатывается не за один шаг; сеть выполняет внутренний цикл, перебирая последовательность элементов. _RNN_ это цикл _for_, который повторно использует величины, вычисленные в предыдущей итерации.

<center><img src="3.1.5.png"></center>
<center>Рис. 2. Рекуррентная сеть — сеть с циклом</center>

Согласно варианту используем рекурретный слой GRU. GRU (Gated Recurrent Unit), является одним из видов рекуррентных нейронных сетей (RNN), аналогичным LSTM (Long Short-Term Memory). GRU решает проблему с затуханием градиента. Суть работы слоя LSTM: он сохраняет информацию для последующего использования, тем самым предотвращая постепенное затухание старых сигналов во время обработки. Слои управляемых рекуррентных блоков GRU основаны на том же принципе, что и слои LSTM, однако они представляют собой более простые структуры и, соответственно, менее затратны в вычислительном смысле (хотя могут не иметь такой же репрезентативной мощности, как LSTM). Этот компромисс между затратностью вычислений и репрезентативной мощностью можно наблюдать повсюду в области машинного обучения.

<center><img src="3.1.77.png"></center>
<center>Рис. 3. Работа ячейки GRU</center>

Здесь $u_t$ — это гейт обновления (update gate), который и является комбинацией
входного и забывающего гейтов. А $r_t$ — это гейт перезагрузки (reset gate); он тоже
отвечает за то, какую часть памяти нужно перенести дальше с прошлого шага, но
делает это еще до применения нелинейной функции. Ячейка памяти и выход блока
$h_t$ тут, в отличие от LSTM, никак не разделяются, и следующий выход $h_t$ получается как комбинация (задаваемая гейтом $u_t$) предыдущего выхода $h_t−1$ и текущего
кандидата в выход $h^{′}_t$ , который, в свою очередь, тоже зависит от $h_t−1$, но на этот раз через гейт перезагрузки $r_t$.

<center><img src="3.1.8.png"></center>
<center>Рис. 4. Структура ячейки GRU</center>

Основная разница между GRU и LSTM состоит в том, что GRU пытается сделать двумя гейтами то же самое, что LSTM делает тремя. Обязанности забывающего гейта f в LSTM здесь разделены между двумя гейтами, t и $u_t$. Кроме того, не возникает второй нелинейности на пути от входа к выходу, как в случае LSTM. Заметим еще, что здесь опять нужно правильно проинициализировать свободные члены в гейте обновления $u_t$: свободные члены $b_u$ должны быть большими, иначе опять возникнет нежелательный эффект с экспоненциальным затуханием «памяти» в последовательности GRU.

In [8]:
def create_model():
    model = Sequential()
    model.add(Embedding(max_features, 32, input_length = maxlen))
    model.add(GRU(128))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
    return model

Перекрестная энтропия (crossentropy) - это мера расстояния между распределениями вероятностей, или в данном случае - между фактическими данными и предсказаниями.

Настраиваем модель оптимизатором rmsprop и функцией потерь mеап squared error.

rmsprop - наиболее подходящий оптимизатор, популярный в использовании для большинства нейронных сетей.

Среднеквадратичное распространение корня (RMSprop) - это экспоненциально затухающее среднее значение. Существенным свойством RMSprop является то, что вы не ограничены только суммой прошлых градиентов, но вы более ограничены градиентами последних временных шагов. В RMSProp мы пытаемся уменьшить вертикальное движение, используя среднее значение, потому что они суммируются приблизительно до 0, принимая среднее значение. RMSprop предоставляет среднее значение для обновления. Формула обновления изображена на рисунке 5.

<center><img src="2.1.8.png"></center>
<center>Рис. 5. Формула rmsprop</center>

    7. Проведение проверки решения, выделяя контрольное множество:

In [9]:
model = create_model()
history = model.fit(input_train, y_train,
    epochs=60,
    batch_size=128,
    validation_split=0.2)



Epoch 1/60


157/157 [==============================] - 78s 488ms/step - loss: 0.6574 - acc: 0.6047 - val_loss: 0.5283 - val_acc: 0.7460
Epoch 2/60
157/157 [==============================] - 79s 502ms/step - loss: 0.4270 - acc: 0.8051 - val_loss: 0.4374 - val_acc: 0.8042
Epoch 3/60
157/157 [==============================] - 79s 503ms/step - loss: 0.3403 - acc: 0.8615 - val_loss: 0.3838 - val_acc: 0.8268
Epoch 4/60
157/157 [==============================] - 79s 504ms/step - loss: 0.2969 - acc: 0.8794 - val_loss: 0.3443 - val_acc: 0.8660
Epoch 5/60
157/157 [==============================] - 80s 509ms/step - loss: 0.2470 - acc: 0.9032 - val_loss: 0.3436 - val_acc: 0.8500
Epoch 6/60
157/157 [==============================] - 80s 510ms/step - loss: 0.2221 - acc: 0.9159 - val_loss: 0.3426 - val_acc: 0.8572
Epoch 7/60
157/157 [==============================] - 78s 494ms/step - loss: 0.1995 - acc: 0.9233 - val_loss: 0.3953 - val_acc: 0.8338
Epoch 8/60
157/157 [==============================]

    8. Вывод графиков функции потерь и точности:

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

<center><img src="3.1.1.png"></center>
<center>Рис. 6. Точность на этапах обучения и проверки</center>
<center><img src="3.1.2.png"></center>
<center>Рис. 7. Потери на этапах обучения и проверки</center>
    
    9. Использование обученной сети для предсказания на новых данных:

In [11]:
results = model.evaluate(input_test, y_test)

782/782 [==============================] - 29s 37ms/step - loss: 1.0218 - acc: 0.8314


По итогам обучения с 60 эпохами точность достигла 83%.

На этапе обучения потери снижаются с каждой эпохой, а точность растет. Это ожидаемое поведение от оптимизации градиентным спуском: величина, которую необходимо минимизировать, становиться все меньше с каждой итерацией на обучающих данных. Однако потери и точность на этапе проверки достигает пика в третью эпоху. Далее происходит переобучение: после третьей эпохи произошла чрезмерная оптимизация на обучающих данных, и в результате получилось представление, характерное для обучающих данных, не обобщающее данные за пределами обучающего набора.

В данном случае для предотвращения переобучения можно прекратить обучение после 8 эпохи.

Скомпилируем заново модель, ограничимся 8 эпохами и проверим точность на контрольных данных:

In [12]:
model = create_model()
model.fit(input_train, y_train,
    epochs=8,
    batch_size=128,
    validation_split=0.2)

Epoch 1/8
157/157 [==============================] - 78s 489ms/step - loss: 0.6336 - acc: 0.6154 - val_loss: 0.5393 - val_acc: 0.7300
Epoch 2/8
157/157 [==============================] - 74s 474ms/step - loss: 0.4239 - acc: 0.8128 - val_loss: 0.4447 - val_acc: 0.8040
Epoch 3/8
157/157 [==============================] - 75s 477ms/step - loss: 0.3551 - acc: 0.8594 - val_loss: 0.3586 - val_acc: 0.8564
Epoch 4/8
157/157 [==============================] - 75s 478ms/step - loss: 0.2969 - acc: 0.8791 - val_loss: 0.3844 - val_acc: 0.8326
Epoch 5/8
157/157 [==============================] - 75s 477ms/step - loss: 0.2532 - acc: 0.9004 - val_loss: 0.3913 - val_acc: 0.8332
Epoch 6/8
157/157 [==============================] - 76s 482ms/step - loss: 0.2221 - acc: 0.9160 - val_loss: 0.3232 - val_acc: 0.8754
Epoch 7/8
157/157 [==============================] - 75s 478ms/step - loss: 0.2008 - acc: 0.9238 - val_loss: 0.3882 - val_acc: 0.8638
Epoch 8/8
157/157 [==============================] - 76s 481ms

In [14]:
results_end = model.evaluate(input_test, y_test)

782/782 [==============================] - 28s 36ms/step - loss: 0.3458 - acc: 0.8548


Модель обученная в течении 8 эпох показала точность в 85.5% при потерях 0.3458.

Посмотрим сколько мы выигрываем при обучении модели на двадцати эпохах и модели обученной на трех эпохах:

In [15]:
print(results_end[0] - results[0])
print(results_end[1] - results[1])

-0.6759388744831085
0.023320019245147705


Как можно увидеть, мы выйграли не только время обучения сети, но и 0.02 точности и 0.68 потери. 
    
    10. Сопоставление полученных результатов с одномерной сверточной сетью (три сверточных слоя).

Для оценки качества работы модели на основе реккурентной сети можем использовать сравнение с одномерной сверточной сетью.

Одномерные сверточные нейронные сети могут состязаться с рекуррентными сетями в некоторых задачах обработки последовательностей, как правило, требуя меньше вычислительных ресурсов. Небольшие одномерные сверточные нейронные сети могут служить быстрой альтернативой рекуррентным сетям в простых задачах, таких как классификация текста и прогнозирование временных последовательностей. 

Одномерные сверточные слои способны распознавать локальные шаблоны в последовательности. Поскольку к каждому шаблону применяются одни и те же преобразования, тот или иной шаблон, найденный в некоторой позиции в предложении, позднее может быть опознан в другой позиции, что делает преобразования, выполняемые одномерными сверточными сетями, инвариантными (во времени)

<center><img src="3.1.9.png"></center>
<center>Рис. 8. Иллюстрации принципа одномерной свертки</center>

С помощью функции _Sequential()_ создаём новую модель. Первый слой, _Embedding(num_words, 32, input_length=maxlen)_, слой встраивания, который преобразует индексы слов в dense-векторы размерности 32, аналогичен с предыдущей модели. Второй слой Conv1D с 32 фильтрами размера 7 и функцией активации _relu_. Этот слой извлекает локальные признаки из входных векторов. Затем добавлен MaxPooling1D(5): слой максимального объединения, который уменьшает размерность выходных данных сверточного слоя путем выбора максимального значения из окна размера 5. Следующий слой анологичен со вторым слоем, который извлекает более высокоуровневые признаки из текстовых данных. Предпоследний слой _GlobalMaxPooling1D()_, слой глобального максимального объединения, который выбирает максимальное значение из каждого канала, что приводит к одному вектору признаков для всего входного текста. Последний слой состоит из одного нейрона с функцией активации _sigmoid_, на выходе получаем скалярное значение в диапозоне между 0 и 1, представляющее собой вероятность. Он выдает вероятность принадлежности входного текста к положительному классу.

После определения архитектуры модель компилируется с помощью функции model.compile(). Используется оптимизатор RMSprop для обновления весов модели во время обучения. Функция потерь бинарной кросс-энтропии, подходящая для задачи бинарной классификации. Метрика точности будет отслеживаться во время обучения.

In [7]:
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(Dense(1))
model.compile(optimizer=RMSprop(lr=1e-4),
    loss='binary_crossentropy',
    metrics=['acc'])
history2 = model.fit(input_train, y_train,
    epochs=10,
    batch_size=128,
    validation_split=0.2)

Epoch 1/10



157/157 [==============================] - 8s 44ms/step - loss: 7.7364 - acc: 0.4985 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 2/10
157/157 [==============================] - 7s 42ms/step - loss: 7.7364 - acc: 0.4985 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 3/10
157/157 [==============================] - 6s 40ms/step - loss: 7.7364 - acc: 0.4985 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 4/10
157/157 [==============================] - 6s 41ms/step - loss: 7.7364 - acc: 0.4985 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 5/10
157/157 [==============================] - 7s 43ms/step - loss: 7.7364 - acc: 0.4985 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 6/10
157/157 [==============================] - 6s 40ms/step - loss: 7.7364 - acc: 0.4985 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 7/10
157/157 [==============================] - 6s 39ms/step - loss: 7.7364 - acc: 0.4985 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 8/10
157/157 [==============================] - 6s 37ms/step - loss: 7.736

In [24]:
results_end3 = model.evaluate(input_test, y_test)

782/782 [==============================] - 3s 4ms/step - loss: 1.0862 - acc: 0.8646


Выведем графики потери и точности:

In [ ]:
acc = history2.history['acc']
val_acc = history2.history['val_acc']
loss = history2.history['loss']
val_loss = history2.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

<center><img src="3.1.3.png"></center>
<center>Рис. 9. Точность на этапах обучения и проверки</center>
<center><img src="3.1.4.png"></center>
<center>Рис. 10. Потери на этапах обучения и проверки</center>

По итогам обучения с 10 эпохами точность достигла 86%.

Первая модель показывает точность 85% при потерях в 0.34, вторая 86% при потерях в 1. Как видно точность прогнозирования почти индентичны, но потери одномерной сверточной сети значительно больше. 

Пример работы обученной модели:

In [5]:
index = imdb.get_word_index()

reverse_index = dict([(value,key) for (key, value) in index.items()])
decoder = ' '.join([reverse_index.get(i-3,'.') for i in input_test[0]])
print(decoder)

. please give this one a miss br br . . and the rest of the cast rendered terrible performances the show is flat flat flat br br i don't know how michael madison could have allowed this one on his plate he almost seemed to know this wasn't going to work out and his performance was quite . so all you madison fans give this a miss


In [8]:
def rating(x):
    if x>0.5:
        return "положительный отзыв"
    else:
        return "отрицательный отзыв"

In [9]:
ex = input_test[0]
ex = np.expand_dims(ex, axis=0)
pred = model.predict(ex)[0]
print('Predict: ',rating(pred))
print('True: ',rating(y_test[0]))

1/1 [==============================] - 0s 80ms/step
Predict:  отрицательный отзыв
True:  отрицательный отзыв


Модель первый из тестовых отзывов оценила как отрицательный отзыв. Провелили в списках меток. Они совпали

## Этап 2. Построение прогноза температуры с использованием рекуррентных и сверточных нейронных сетей.

**Целью данного этапа** лабораторной работы является создание нейронной сети, реализующей прогноз температуры по набору данных Jena Climate.

Вариант этапа 2
| № | Количество рекуррентных слоев | Количество нейронов на слое | Рекуррентный слой | Прореживание |
| :-: | :-: | :-: | :-: | :-: |
| 3 | 2 | 64-32 | GRU (двунаправленная сеть) | 0.2 |

 Пошаговая реализация поставленной цели включает:

1. Загрузка набора данных Jena Climate:

В набор данных включены замеры 14 разных характеристик (таких, как температура, атмосферное давление, влажность, направление ветра и т. д.), выполнявшиеся каждые 10 минут в течение нескольких лет,  в этот пример включены только данные за 2009–2016 годы. Данные изначально не разделены на тренировочные и тестовые наборы. Загрузим данные:

In [2]:
data_dir = "C:/Users/svyatoslav/Desktop/магистратура/2 курс 4 семестр/Спецкурс по Нейронным сетям/jena_climate_2009_2016.csv"
fname = os.path.join(data_dir, 'jena_climate_2009_2016.csv')
f = open(fname)
data = f.read()
f.close()
lines = data.split('\n')
header = lines[0].split(',')
lines = lines[1:]

Преобразуем данные в массив Numpy:

In [3]:
float_data = np.zeros((len(lines), len(header) - 1))
for i, line in enumerate(lines):
 values = [float(x) for x in line.split(',')[1:]]
 float_data[i, :] = values

Для предварительного анализа и понимания характера данных перед их использованием для обучения нейронной сети можно осуществить визуализацию временных рядов признаков. 
Визуализация позволяет быстро оценить:
*	Наличие пропущенных значений или выбросов в данных.
*	Характер распределения признаков (периодичность, тренды, сезонность и т.д.).
*	Диапазоны значений для различных признаков.
*	Взаимосвязь между признаками (если они коррелируют, то графики могут иметь схожий характер).

Визуализируем временные ряды:

In [ ]:
keys_grafs = ["p (mbar)","T (degC)","Tpot (K)","Tdew (degC)","rh (%)","VPmax (mbar)","VPact (mbar)","VPdef (mbar)","sh (g/kg)","H2OC (mmol/mol)","rho (g/m**3)","wv (m/s)","max. wv (m/s)","wd (deg)"]
for i in range(len(keys_grafs)):
    temp = float_data[:, i] # температура (в градусах Цельсия)
    plt.plot(range(len(temp)), temp)
    plt.title(keys_grafs[i]) 
    plt.show()

<center><img src="3.2.1.png"></center>
<center><img src="3.2.2.png"></center>
<center><img src="3.2.3.png"></center>
<center><img src="3.2.4.png"></center>
<center><img src="3.2.5.png"></center>
<center><img src="3.2.6.png"></center>
<center><img src="3.2.7.png"></center>
<center><img src="3.2.8.png"></center>
<center><img src="3.2.9.png"></center>
<center><img src="3.2.10.png"></center>
<center><img src="3.2.11.png"></center>
<center><img src="3.2.12.png"></center>
<center><img src="3.2.13.png"></center>
<center><img src="3.2.14.png"></center>
<center>Рис. 11. Графики зависимости признаков от времени</center>

Анализируя данные графики можем сделать следующие заключения:
*	Многие признаки демонстрируют явную сезонность и периодические колебания, что характерно для климатических данных.
*	Разные признаки имеют различные диапазоны значений. 

    Это указывает на необходимость нормализации или масштабирования данных перед обучением модели машинного обучения.
*	Можно заметить несколько потенциальных выбросов, особенно в данных о скорости ветра и давлении.
*	Некоторые признаки, такие как температура, точка росы и влажность, демонстрируют схожие паттерны, что может указывать на их взаимосвязь и корреляцию.
*	Многие признаки демонстрируют явную годовую периодичность, что соответствует сезонным изменениям климата.

    3. Создание генератора данных, выбрав базу данных за 10 дней, задержку в 1 день, 1 значение в час и размер пакета из варианта:

_generator_ — функция-генератор данных. Она возвращает кортеж (образцы, цели), где образцы — это один пакет входных данных, а цели — соответствующий массив целевых температур. Функция принимает следующие аргументы:

* data — исходный массив вещественных чисел, который будет нормализоваться дальше;
* lookback — количество интервалов в прошлом от заданного момента, за которое отбираются входные данные;
* delay — количество интервалов в будущем от заданного момента, за которое отбираются целевые данные;
* min_index и max_index — индексы в массиве data, ограничивающие область для извлечения данных; это помогает оставить в неприкосновенности сегменты проверочных и контрольных данных;
* shuffle — флаг, определяющий порядок извлечения образцов: с перемешиванием или в хронологическом порядке;
* batch_size — количество образцов в пакете;
* step — период в интервалах, из которого извлекается один образец; мы установим его равным 6, чтобы получить по одному образцу за каждый час.

In [4]:
def generator(data, lookback, delay, min_index, max_index,
    shuffle=False, batch_size=128, step=6):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(
                min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)
        samples = np.zeros((len(rows),
                            lookback // step,
                            data.shape[-1]))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]
        yield samples, targets

    4. Подготовка данных для передачи в нейронную сеть:

В процессе обработки данных мы будем вычитать среднее для каждой временной последовательности и делить на стандартное отклонение. Для обучения мы используем первые 200000 замеров, поэтому среднее и стандартное отклонение должны вычисляться только по этой выборке.

In [5]:
mean = float_data[:200000].mean(axis=0)
float_data -= mean
std = float_data[:200000].std(axis=0)
float_data /= std

Используя функцию-генератор _generator_ для создания трех других функций-генераторов (train_gen, val_gen и test_gen): для получения обучающих, проверочных и контрольных данных. Все они будут отбирать образцы из разных временных сегментов оригинальных данных: обучающие данные будут извлекаться из первых 200 000 интервалов, проверочные — из следующих 100 000, а контрольные — из остальных. бучающий генератор (train_gen) использует данные с индексами от 0 до 200000. Валидационный генератор (val_gen) использует данные с индексами от 200001 до 300000. Тестовый генератор (test_gen) использует данные с индексами после 300001. Переменные val_steps и test_steps вычисляют количество шагов (итераций) для валидационного и тестового генераторов соответственно.

* lookback — определяет длину истории (временной ряд из прошлых значений);
* delay — смещение до целевого значения;
* step — управляет шагом выборки данных из временного ряда. 
* shuffle — определяет, нужно ли перемешивать данные перед выборкой. 
* batch_size — задает размер порции данных, возвращаемой генератором за один вызов.

In [6]:
lookback = 1440
step = 6
delay = 144
batch_size = 128
train_gen = generator(float_data,
    lookback=lookback,
    delay=delay,
    min_index=0,
    max_index=200000,
    shuffle=True,
    step=step,
    batch_size=batch_size)
val_gen = generator(float_data,
    lookback=lookback,
    delay=delay,
    min_index=200001,
    max_index=300000,
    step=step,
    batch_size=batch_size)
test_gen = generator(float_data,
    lookback=lookback,
    delay=delay,
    min_index=300001,
    max_index=None,
    step=step,
    batch_size=batch_size)
val_steps = (300000 - 200001 - lookback) // batch_size
test_steps = (len(float_data) - 300001 - lookback) // batch_size

    5. Выполнить конструирование сети, создание сети рекуррентных слоев заданного типа в соответствии с вариантом:

С помощью функции _Sequential()_ создаём новую модель. Первый слой, _Bidirectional_, слой встраивания, который в своем первом аргументе принимает экземпляр рекуррентного
слоя _GRU_ с 64 нейронами, получает на вход временные последовательности данных с любым количеством временных шагов _(None)_ и размерностью признаков, равной размерности исходных данных _(values.shape[-1])_, параметры _dropout_ и _recurrent_dropout_ применяют регуляризацию отсечением для предотвращения переобучения, параметр _return_sequences_ указывает, что слой должен возвращать полную последовательность выходов, а не только выход на последнем временном шаге. Слой Bidirectional создает второй, отдельный экземпляр этого рекуррентного слоя и использует один экземпляр для обработки входных последовательностей в прямом порядке, а другой — в обратном. Второй слой _Bidirectional_ такой же как и первый, только рекуррентный слой GRU c 32 нейронами, он не возвращает полную последовательность выходов, а только выход на последнем временном шаге. Последний сдлой идет полносвязный слой (Dense) с одним нейроном для выполнения задачи регрессии (предсказания одного целевого значения).

Слой Bidirectional создает второй, отдельный экземпляр этого рекуррентного слоя и использует один экземпляр для обработки входных последовательностей
в прямом порядке, а другой — в обратном.

В RNN есть ещё один тип сетей — это двунаправленные рекуррентные сети. Часто бывает так, что RNN к концу последовательности уже забывают, с чего все начиналось, также последние элементы последовательности, даже если не забудем начало, всегда будут гораздо важнее первых и в обычной RNN, и в сети из LSTM или GRU-ячеек. Поэтому часто рассматривают так называемые двунаправленные рекуррентные сети (bidirectional RNN). 

Проиллюстрируем на рис. 11 структуру двунаправленной нейронной сети. На этой схеме мы «спрятали» матрицы $W$ и $U$ в один блок и сконцентрировались на том, чтобы детально показать, что происходит с выходами; связи, относящиеся ко идущей справа налево рекуррентной сети, на рис. 11 показаны пунктиром. Формально говоря, в двунаправленной сети мы вычисляем состояния $s_t$ слева направо и состояния $s^{′}_t$ справа налево, а затем сливаем их в один результат уже на уровне выхода; это значит, что выход вычисляется как

<center><img src="3.2.16.png"></center>
<center>Рис. 12. Вычисление выхода из двунаправленной рекуррентной сети</center>

<center><img src="3.2.115.png"></center>
<center>Рис. 13. Двунаправленная рекуррентная сеть</center>

Двунаправленная рекуррентная сеть использует чувствительность RNN к порядку: она состоит из двух обычных рекуррентных сетей, таких как слои GRU и LSTM. Обрабатывая последовательность в двух направлениях, двунаправленная рекуррентная сеть способна выявить шаблоны, незаметные для однонаправленной сети.  Она просматривает входную последовательность в обоих направлениях (рис. 14), получает потенциально более насыщенные представления и выделяет шаблоны, которые могли быть упущены однонаправленной версией. Двунаправленные рекуррентные сети, просматривающие последовательность данных в обоих направлениях, дают хорошие результаты в задачах обработки естественного языка. Но они мало пригодны для обработки последовательностей, в которых недавние данные информативнее, чем находящиеся в начале.

<center><img src="3.2.17.png"></center>
<center>Рис. 14. Принцип действия двунаправленной рекуррентной нейронной сети</center>

In [48]:
model = Sequential()
model.add(layers.Bidirectional(layers.GRU(64, return_sequences=True, recurrent_dropout=0.2, dropout=0.2), input_shape=(None, float_data.shape[-1])))
model.add(layers.Bidirectional(layers.GRU(32, recurrent_dropout=0.2, dropout=0.2)))
model.add(layers.Dense(1))

    6. Настройка оптимизатора с выбором функции потерь и метрики качества. Число эпох принять от 30 до 60

Скомпелировали модель с помощью функции _compile_. Модель компилируется с использованием функции потерь mae (среднее абсолютное отклонение) и оптимизатора RMSprop. Функция потерь mae вычисляет среднее абсолютное отклонение между предсказанными и истинными значениями, что подходит для задач регрессии. Оптимизатор RMSprop (Root Mean Square Propagation) представляет собой адаптивный метод оптимизации, который регулирует скорость обучения в зависимости от истории градиентов.

In [49]:
model.compile(optimizer=RMSprop(), loss='mae')

    7. Проведение проверки решения, выделяя контрольное множество:

Обучим модель на 30 эпохах с шагом валидации в 500:

In [50]:
history = model.fit(train_gen,
 steps_per_epoch=500,
 epochs=30,
 validation_data=val_gen,
 validation_steps=val_steps)

Epoch 1/30
500/500 [==============================] - 850s 2s/step - loss: 0.3133 - val_loss: 0.2760
Epoch 2/30
500/500 [==============================] - 823s 2s/step - loss: 0.2924 - val_loss: 0.2776
Epoch 3/30
500/500 [==============================] - 825s 2s/step - loss: 0.2850 - val_loss: 0.2753
Epoch 4/30
500/500 [==============================] - 840s 2s/step - loss: 0.2784 - val_loss: 0.2810
Epoch 5/30
500/500 [==============================] - 843s 2s/step - loss: 0.2723 - val_loss: 0.2737
Epoch 6/30
500/500 [==============================] - 846s 2s/step - loss: 0.2662 - val_loss: 0.2946
Epoch 7/30
500/500 [==============================] - 855s 2s/step - loss: 0.2579 - val_loss: 0.2867
Epoch 8/30
500/500 [==============================] - 845s 2s/step - loss: 0.2493 - val_loss: 0.2925
Epoch 9/30
500/500 [==============================] - 843s 2s/step - loss: 0.2411 - val_loss: 0.2951
Epoch 10/30
500/500 [==============================] - 838s 2s/step - loss: 0.2329 - val_lo

    8. Вывод графиков функции потерь и точности:

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

<center><img src="3.2.18.png"></center>
<center>Рис. 15. Потери на этапах обучения и проверки в задаче прогнозирования температуры по данным Jena</center>

После 3 эпохи наблюдается постепенное расхождение между значениями функции потерь для обучающего и валидационного наборов. Это указывает на переобучение модели. В данном случае для предотвращения переобучения можно прекратить обучение после 3 эпохи.

Скомпилируем заново модель, ограничимся 3 эпохами:

In [40]:
model = Sequential()
model.add(layers.Bidirectional(layers.GRU(64, return_sequences=True, recurrent_dropout=0.2, dropout=0.2), input_shape=(None, float_data.shape[-1])))
model.add(layers.Bidirectional(layers.GRU(32, recurrent_dropout=0.2, dropout=0.2)))
model.add(layers.Dense(1))
model.compile(optimizer=RMSprop(), loss='mae')
model.fit(train_gen,
 steps_per_epoch=500,
 epochs=5,
 validation_data=val_gen,
 validation_steps=val_steps)

Epoch 1/3
500/500 [==============================] - 1035s 2s/step - loss: 0.3136 - val_loss: 0.2699
Epoch 2/3
500/500 [==============================] - 1073s 2s/step - loss: 0.2929 - val_loss: 0.2834
Epoch 3/3
500/500 [==============================] - 877s 2s/step - loss: 0.2829 - val_loss: 0.2762


    9. Использование обученной сети для предсказания на новых данных:

In [41]:
model.evaluate(test_gen, steps=100)

100/100 [==============================] - 10s 99ms/step - loss: 0.2603


0.2602558732032776

Итоговое значение функции потерь на тестовых данных составляет 0.26.

    10. Сопоставление полученных результатов с сетью, первый блок которого является одномерной сверточной сетью (три сверточных слоя), а затем идет второй блок, указанный в варианте:

С помощью функции _Sequential()_ создаём новую модель. Первый слой Conv1D с 32 фильтрами размера 7 и функцией активации _relu_, получает на вход временные последовательности данных с любым количеством временных шагов _(None)_ и размерностью признаков, равной размерности исходных данных _(values.shape[-1])_. Этот слой извлекает локальные признаки из входных векторов. Затем добавлен MaxPooling1D(5): слой максимального объединения, который уменьшает размерность выходных данных сверточного слоя путем выбора максимального значения из окна размера 5. Следующий слой анологичен со вторым слоем, который извлекает более высокоуровневые признаки из текстовых данных. После свёрточных слоев добавим слои, которые были в предыдущей модели: добавим слой _Bidirectional_, слой встраивания, который в своем первом аргументе принимает экземпляр рекуррентного слоя _GRU_ с 64 нейронами, параметры _dropout_ и _recurrent_dropout_ применяют регуляризацию отсечением для предотвращения переобучения, параметр _return_sequences_ указывает, что слой должен возвращать полную последовательность выходов, а не только выход на последнем временном шаге. Слой Bidirectional создает второй, отдельный экземпляр этого рекуррентного слоя и использует один экземпляр для обработки входных последовательностей в прямом порядке, а другой — в обратном. Следующий слой _Bidirectional_ такой же как и предыдущий, только рекуррентный слой GRU c 32 нейронами, он не возвращает полную последовательность выходов, а только выход на последнем временном шаге. Последний слой идет полносвязный слой (Dense) с одним нейроном для выполнения задачи регрессии (предсказания одного целевого значения).

Скомпелировали модель с помощью функции _compile_. Модель компилируется с той же функции потери mae и оптимизатором RMSprop.

Обучение модели производилось в течение 60 эпох, с шагом валидации в 500:

In [44]:
model = Sequential()
model.add(layers.Conv1D(32, 7, activation='relu', input_shape=(None, float_data.shape[-1])))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.Bidirectional( layers.GRU(64, return_sequences=True, recurrent_dropout=0.2, dropout=0.2)))
model.add(layers.Bidirectional( layers.GRU(32, recurrent_dropout=0.2, dropout=0.2)))
model.add(layers.Dense(1))
model.compile(optimizer=RMSprop(), loss='mae')
history = model.fit(train_gen,
 steps_per_epoch=500,
 epochs=60,
 validation_data=val_gen,
 validation_steps=val_steps)

Epoch 1/60
500/500 [==============================] - 63s 111ms/step - loss: 0.3481 - val_loss: 0.3097
Epoch 2/60
500/500 [==============================] - 53s 107ms/step - loss: 0.2996 - val_loss: 0.3324
Epoch 3/60
500/500 [==============================] - 54s 109ms/step - loss: 0.2782 - val_loss: 0.3300
Epoch 4/60
500/500 [==============================] - 54s 108ms/step - loss: 0.2615 - val_loss: 0.3360
Epoch 5/60
500/500 [==============================] - 54s 108ms/step - loss: 0.2469 - val_loss: 0.3391
Epoch 6/60
500/500 [==============================] - 69s 138ms/step - loss: 0.2335 - val_loss: 0.3476
Epoch 7/60
500/500 [==============================] - 72s 144ms/step - loss: 0.2226 - val_loss: 0.3518
Epoch 8/60
500/500 [==============================] - 75s 151ms/step - loss: 0.2098 - val_loss: 0.3737
Epoch 9/60
500/500 [==============================] - 74s 149ms/step - loss: 0.2030 - val_loss: 0.3648
Epoch 10/60
500/500 [==============================] - 75s 150ms/step - l

Построение графика функций потерь в течение обучения:

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

<center><img src="3.2.19.png"></center>
<center>Рис. 16. Потери на этапах обучения и проверки в задаче прогнозирования температуры по данным Jena</center>

На первой эпохе значение функции потерь для обучающего набора составляет 0.3481, а для валидационного набора 0.3097. По мере обучения значения функции потерь для обучающего набора постепенно уменьшаются, что указывает на улучшение модели на обучающих данных, однако значения функции потерь для валидационного набора не демонстрируют снижения. Поскольку значения функции потерь для валидационного набора не снижаются, а увеличиваются- это указывает на переобучение модели. 

Скомпилируем заново модель, ограничимся 1 эпохой:

In [47]:
model = Sequential()
model.add(layers.Conv1D(32, 7, activation='relu', input_shape=(None, float_data.shape[-1])))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.Bidirectional( layers.GRU(64, return_sequences=True, recurrent_dropout=0.2, dropout=0.2)))
model.add(layers.Bidirectional( layers.GRU(32, recurrent_dropout=0.2, dropout=0.2)))
model.add(layers.Dense(1))
model.compile(optimizer=RMSprop(), loss='mae')
history = model.fit(train_gen,
 steps_per_epoch=500,
 epochs=1,
 validation_data=val_gen,
 validation_steps=val_steps)
model.evaluate(test_gen, steps=100)

100/100 [==============================] - 2s 15ms/step - loss: 0.3563


0.3562968075275421

В итоге, получили 2 модели показывающие результаты: первая показывает потери в 0.26, а вторая потери в 0.356. 

В сравнении с первой моделью значение функции потерь выше, что указывает на избыточно сложную модель для приведенной задачи, а также показывает слабость выделения признаков из больших временных рядов простыми сверточными моделями. 

<p style="text-align: center;">Заключение</p>

1. На первом этапе мы классифицировали отзывы к фильмам на положительные и отрицательные опираясь на эмоциональные окраски отзывов. Использовали набор данных IMDB с 50000 отзывами. Подготовили данные для передачи в сеть с помощью функции _pad_sequences_, т.е. преобразовали списки целых чисел в двумерный тензор с целыми числами и с формой (образцы, максимальная_длина). Сконструировали 2 сети согласно варианту. Превая сеть RNN состоит из первого слоя, _Embedding(num_words, 32, input_length=maxlen)_, слой встраивания. Второго слоя _GRU_, рекуррентный слой c 128 нейронами. Последний слой состоит из одного нейрона с функцией активации _sigmoid_, на выходе получаем скалярное значение в диапозоне между 0 и 1, представляющее собой вероятность. Вторая одномерные сверточная нейронная сеть состоит из первого слоя, _Embedding(num_words, 32, input_length=maxlen)_, слой встраивания, аналогичен с предыдущей модели. Второго слоя Conv1D с 32 фильтрами размера 7 и функцией активации _relu_. Затем добавлен MaxPooling1D(5). Следующий слой анологичен со вторым слоем, который извлекает более высокоуровневые признаки из текстовых данных. Предпоследний слой _GlobalMaxPooling1D()_, слой глобального максимального объединения. Последний слой состоит из одного нейрона с функцией активации _sigmoid_, на выходе получаем скалярное значение в диапозоне между 0 и 1, представляющее собой вероятность. Он выдает вероятность принадлежности входного текста к положительному классу. В итоге, получили и сравнили 2 модели показывающие результаты: первая 85% при потерях в 0.34, вторая 86% при потерях в 1. Как видно точность прогнозирования почти индентичны, но потери одномерной сверточной сети значительно больше.

2. На втором этапе мы сделали прогноз температуры по набору данных Jena Climate. В этот набор данных были включены замеры 14 разных характеристик. Изучили принцип работы с временными рядами. Обработали данные с помощью нормализации. Познакомились с  двунаправленной рекуррентной нейронной сетью. Сконструировали 2 сети согласно варианту. Превая сеть RNN состоит из первого слоя, _Bidirectional_, слой встраивания, который в своем первом аргументе принимает экземпляр рекуррентного слоя _GRU_ с 64 нейронами. Второй слой _Bidirectional_ такой же как и первый, только рекуррентный слой GRU c 32 нейронами. Последний слой полносвязный слой Dense с одним нейроном для выполнения задачи регрессии (предсказания одного целевого значения). Вторая сеть первый блок которого является одномерной сверточной сетью (три сверточных слоя), а затем идет второй блок, указанный в варианте. Первый слой Conv1D с 32 фильтрами размера 7 и функцией активации _relu_, получает на вход временные последовательности данных с любым количеством временных шагов _(None)_ и размерностью признаков, равной размерности исходных данных _(values.shape[-1])_. Затем добавлен MaxPooling1D(5): слой максимального объединения. Следующий слой анологичен с первым слоем. После свёрточных слоев добавим слои, которые были в 1 модели: Последний слой идет полносвязный слой (Dense) с одним нейроном для выполнения задачи регрессии (предсказания одного целевого значения). В итоге, получили 2 модели показывающие результаты: первая показывает потери в 0.26, а вторая потери в 0.356. В сравнении с первой моделью значение функции потерь выше, что указывает на избыточно сложную модель для приведенной задачи, а также показывает слабость выделения признаков из больших временных рядов простыми сверточными моделями. 

<p style="text-align: center;">Список использованной литературы</p>

1. Шолле Франсуа. Глубокое обучение на Python. - СПб.: Питер, 2018. - 400 с.: ил. - (Серия «Библиотека программиста»).
2. Николенко С., Кадурин А., Архангельская Е. Глубокое обучение. — СПб.: Питер, 2018. — 480 с.: ил. — (Серия «Библиотека программиста»).